**ver3는 anylabeling 프로그램에 넣을 수 있는 문법의 json 파일을 생성하는 코드로 짰다.**

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import json
import base64

os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

height = 384
width = 256
num_classes = 13

dataset_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/new/images"
save_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/new/last"
model_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5"

class_colors = {
    'S1': (255, 0, 0), 'L5': (0, 255, 0), 'L4': (0, 0, 255), 'L3': (255, 255, 0),
    'L2': (255, 0, 255), 'L1': (0, 255, 255), 'T12': (100, 100, 255),
    'T11': (100, 255, 100), 'T10': (255, 100, 100), 'T9': (100, 255, 255),
    'latLSpine': (255, 100, 255), 'latSacrum': (255, 255, 100), 'Aorticcalcification': (100, 100, 100)
}

class_names = list(class_colors.keys())

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir(save_path)

model = tf.keras.models.load_model(model_path)
model.summary()

test_x = sorted(glob(os.path.join(dataset_path, "*.jpg")))
print(f"Test Images: {len(test_x)}")

def find_largest_contour_and_create_labelme_shape(class_mask, class_label):
    contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        contour = largest_contour.flatten().tolist()
        points = [[contour[i], contour[i + 1]] for i in range(0, len(contour), 2)]
        shape = {
            "label": class_label,
            "points": points,
            "shape_type": "polygon",
            "flags": {}
        }
        return shape
    return None

def generate_labelme_json(test_x, model, save_path, class_colors, num_classes, dataset_path, height, width):
    create_dir(save_path)
    time_taken = []
    for x_path in tqdm(test_x):
        name = x_path.split("/")[-1]
        x = cv2.imread(x_path, cv2.IMREAD_COLOR)
        x = cv2.resize(x, (width, height))
        x = x / 255.0
        x = np.expand_dims(x, axis=0)
        start_time = time.time()
        prediction = model.predict(x)[0]
        total_time = time.time() - start_time
        time_taken.append(total_time)
        class_indices = np.argmax(prediction, axis=-1)
        class_masks = {class_names[i]: (class_indices == i).astype(np.uint8) * 255 for i in range(num_classes)}
        all_shapes = []
        for class_label, class_mask in class_masks.items():
            if class_label != 'Aorticcalcification':  # 배경 클래스 레이블을 적절히 변경하세요.
                shape = find_largest_contour_and_create_labelme_shape(class_mask, class_label)
                if shape:
                    all_shapes.append(shape)
        original_img = cv2.imread(os.path.join(dataset_path, name))
        original_img = cv2.resize(original_img, (width, height))
        _, encoded_img = cv2.imencode('.jpg', original_img)
        base64_img_data = base64.b64encode(encoded_img).decode('utf-8')
        labelme_json = {
            "version": "4.5.6",
            "flags": {},
            "shapes": all_shapes,
            "imagePath": name,
            "imageData": base64_img_data,
            "imageHeight": height,
            "imageWidth": width
        }
        json_filename = os.path.join(save_path, f"{name.split('.')[0]}.json")
        with open(json_filename, 'w') as json_file:
            json.dump(labelme_json, json_file, ensure_ascii=False, indent=2)
    mean_time = np.mean(time_taken)
    mean_fps = 1 / mean_time if mean_time > 0 else 0
    print(f"Mean Time: {mean_time:.5f} - Mean FPS: {mean_fps:.5f}")

generate_labelme_json(test_x, model, save_path, class_colors, num_classes, dataset_path, height, width)


Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 384, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 384, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 384, 256, 64)         0         ['batch_normalization[0][0]

  0%|          | 0/16 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step


  6%|▋         | 1/16 [00:06<01:35,  6.37s/it]

1/1 [==============================] - 4s 4s/step


...

1/1 [==============================] - 4s 4s/step


 94%|█████████▍| 15/16 [01:15<00:05,  5.03s/it]

1/1 [==============================] - 3s 3s/step


100%|██████████| 16/16 [01:21<00:00,  5.09s/it]

Mean Time: 4.86322 - Mean FPS: 0.20563
